In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import torch
import os
from torch import nn, optim, device, cuda, Tensor, LongTensor, argmax, bincount
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from transformers import BertForSequenceClassification, BertTokenizer, get_linear_schedule_with_warmup

/tmp/ipykernel_18007/3131763294.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/nate/miniconda3/lib/python3.9/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/nate/miniconda3/lib/python3.9/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(

In [2]:
MODEL_FOLDER = 'models/model_bert'
MODEL = f'{MODEL_FOLDER}/bert_state_dict.pth'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
_model_name = "bert-base-multilingual-uncased"

model = torch.load(MODEL, map_location=DEVICE)

model

# model = BertForSequenceClassification.from_pretrained(
#     _model_name,
#     num_labels=2,
# )
# model.load_state_dict(torch.load(MODEL, map_location=DEVICE))
# model.to(DEVICE)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [4]:
tokenizer = BertTokenizer.from_pretrained(_model_name)

In [10]:
def tokenize_and_prepare(texts, tokenizer, max_length, num_examples=2):
    input_ids = []
    attention_masks = []
    examples = []

    for text in texts:
        encoded_dict = tokenizer.encode_plus(
            text,                        # Text to encode.
            add_special_tokens=True,     # Add '[CLS]' and '[SEP]'.
            max_length=max_length,       # Pad or truncate.
            truncation=True,
            padding='max_length',        # Pad to max_length.
            return_attention_mask=True,  # Construct attn. masks.
            return_tensors='pt',         # Return PyTorch tensors.
        )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

        # Collect examples for printing later
        if len(examples) < num_examples:
            examples.append((text, encoded_dict['input_ids'][0], encoded_dict['attention_mask'][0]))

    # Convert lists into tensors
    input_ids = torch.cat(input_ids, dim=0).to(DEVICE)
    attention_masks = torch.cat(attention_masks, dim=0).to(DEVICE)

    return input_ids, attention_masks, examples

In [12]:
input_ids, attention_masks, _ = tokenize_and_prepare(
  [
    'Gago ka putang ina mo',
    'gago Ka tangina mo',
    'I love you so much',
  ],
  tokenizer,
  255,
)

In [13]:
with torch.inference_mode():
  outputs = model(input_ids, attention_mask=attention_masks)
  
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-2.9954,  2.9719],
        [-3.0099,  2.9770],
        [ 1.4062, -1.4658]], device='cuda:0'), hidden_states=None, attentions=None)

In [21]:
softmax_output = torch.nn.functional.softmax(
  outputs.logits,
  dim=1,
)

softmax_output

tensor([[0.0026, 0.9974],
        [0.0025, 0.9975],
        [0.9464, 0.0536]], device='cuda:0')